## ANNDL Final Project: _Jeopardy!_

In [6]:
import csv
import random

In [7]:
# Import (value, question, answer) three-ples from CSV.
data = []
with open("/Users/fiordali/Downloads/JEOPARDY_CSV.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        data.append(row[4:])

random.shuffle(data) # Do I have to avoid shuffling the data/recreating the train and test data sets?

In [77]:
# Split data into two randomized groups: testing and training data.
idx = len(data) // 2

train_set = data[:idx]
test_set = data[idx:]

# Remove dollar value and answer from training set.
train_questions = []

for item in train_set:
    train_questions.append(item[1])

In [76]:
# Find length of longest training question by character.
max_len = 0
counter = 0

for question in train_questions:
    for letter in question:
        counter += 1
    if counter > max_len:
        max_len = counter
    counter = 0

In [ ]:
# Train LSTM on half of questions

In [72]:
# Many-to-one RNN: variable length question maps to fixed length feature vector
sequence = array(train_questions[0])

print(sequence.shape)

sequence.reshape((1, 83, 1))

()


ValueError: cannot reshape array of size 1 into shape (1,83,1)

In [ ]:
# Train FF on feature vectors from LSTM

## References:
* [Understanding LSTMs](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [Emma Boettcher Thesis](https://futurism.com/jeopardy-emma-boettcher-ai-james-holzhauer)
* [A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)
* [LSTM – nuggest for practical application](https://towardsdatascience.com/lstm-nuggets-for-practical-applications-5beef5252092)